In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;


/Users/standarduser/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
train = pd.read_csv('modified_train.csv')
test = pd.read_csv('modified_test.csv')


In [13]:
#Using SVC model
clf = SVC(random_state=0,kernel='linear', C=0.025)


In [7]:
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)


In [8]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [15]:
from sklearn.model_selection import train_test_split
y = train['Survived']
train.drop('Survived',axis=1,inplace= True)
X = train

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.40,random_state=63)

In [20]:
X_train.shape, X_test.shape, len(y_train), len(y_test)

((534, 10), (357, 10), 534, 357)

In [49]:
clf.fit(X_train,y_train)


SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
predict = clf.predict(X_test)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predict)

0.81512605042016806

In [ ]:
# 0.753514 at random_state=12
# 0.77 at random_state= 42
# 0.789915 at random_state = 52 
# 0.8011 at random_state=62
# 0.775 at random_state = 72
# 0.815126 at random_state = 63

In [52]:
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
Unnamed: 0       418 non-null int64
Pclass           418 non-null int64
Sex              418 non-null int64
Embarked         418 non-null int64
FamilySize       418 non-null int64
IsAlone          418 non-null int64
Fare_category    418 non-null int64
Age_category     418 non-null int64
Title            418 non-null int64
Has_Cabin        418 non-null int64
dtypes: int64(10)
memory usage: 32.7 KB


In [53]:
result = clf.predict(test)

In [54]:
actual_test= pd.read_csv('test.csv')

In [57]:
svc_submission = pd.DataFrame({'PassengerId':np.array(actual_test['PassengerId']), 'Survived':np.array(result)})


In [58]:
svc_submission.to_csv('svc_submission.csv')

In [59]:
svc_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
